# Task 2- Variational Classifier

In this notebook, we follow the tutorial on Variational classifiers using Pennylane([here](https://pennylane.ai/qml/demos/tutorial_variational_classifier/)) and explain what we have learnt, through comments within each code block as well as a brief summary in the end of the notebook.

The first example shows that a variational circuit can
be optimized to emulate the parity function

$$\begin{aligned}
f: x \in \{0,1\}^{\otimes n} \rightarrow y =
\begin{cases} 1 \text{  if uneven number of 1's in } x \\ 0
\text{ else}. \end{cases}
\end{aligned}$$

It demonstrates how to encode binary inputs into the initial state of
the variational circuit, which is simply a computational basis state
(*basis encoding*).

The second example shows how to encode real vectors as amplitude vectors
into quantum states (*amplitude encoding*) and how to train a
variational circuit to recognize the first two classes of flowers in the
Iris dataset.

__Contributors:__ Nandan Patel, Phalak Bhatnagar

### EXAMPLE 1: Fitting Parity Function

In [ ]:
#IMPORTS
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [ ]:
#Defining the quantum device with 4 qubits
dev = qml.device("default.qubit", wires=4)

In [ ]:
def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)


In [ ]:
# Defining a Variational Quantum Circuit
# A variational quantum circuit consists of layers of parameterized rotation gates(Rx(theta)) and CNOT gates, theta is determined by weights.
# In the following code, we apply CNOT gates between qubits 0&1, 1&2, 2&3, 0&3
def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)

In [ ]:
# We use Basis State encoding to encode the input values into our qubits,
# For eg. 0101->|0101⟩
def state_preparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])

In [ ]:
# Defines a function to prepare the state preparation circuit, it applies the quantum circuit layers with the given weights
# Outputs an expectation value of the Z operator on the first qubit
@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(0))

In [ ]:
# Adds a bias to the circuit to the output of the quantum circuit to make the model more effective
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [ ]:
# We define a cost function as the root of the difference between the actual labels and the predicted values
def square_loss(labels, predictions):
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    return np.mean((labels - qml.math.stack(predictions)) ** 2)
# We also define the accuracy
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc


In [ ]:
# Defining the cost function
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [ ]:
# Separating the input data into x and y variables, x as input and y as predicted labels,
# The dataset has y values ranging from 0 to 1, we shift it from -1 to 1
data = np.loadtxt(r"C:\Users\dpsso\Desktop\variational_classifier\data\parity_train.txt", dtype=int)
X = np.array(data[:, :-1])
Y = np.array(data[:, -1])
Y = Y * 2 - 1  # shift label from {0, 1} to {-1, 1}

for x,y in zip(X, Y):
    print(f"x = {x}, y = {y}")

In [ ]:
# We initialize the weight values, and fix the number of layers and number of qubits
np.random.seed(0)
num_qubits = 4
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print("Weights:", weights_init)
print("Bias: ", bias_init)

In [ ]:
# We use the NesterovMomentumOptimizer with bias as 0.5 and batch size as 5
# This optimizer is used for enhanced efficiency and navigating the loss surface effectively
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

In [ ]:
# We now execute our cicruit and run our optimizer to train our model
# For each iteration, the cost and the accuracy of the model is calculated and the weights and bias is updated accordingly
weights = weights_init
bias = bias_init
for it in range(100):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")


In [ ]:
# After training our model, we compare the predicted values and the actual y-labels for each set of x values
# This step is important as it checks if our model, which gave excellent results in training, gives the same performance while using unseen test data
data = np.loadtxt(r"C:\Users\dpsso\Desktop\variational_classifier\data\parity_test.txt", dtype=int)
X_test = np.array(data[:, :-1])
Y_test = np.array(data[:, -1])
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)


We get to know that the accuracy was 100% for this example.
The goal of this example was used to accurately predict the output y as the output of the parity function.


### EXAMPLE 2: Iris Classification

In [ ]:
#Here, instead of basis encoding, we use real 2-dimensional vectors
#After adding "latent dimensions(extra dimensions)", we get the total no. of dimensions as 4, therefore we use 2 qubits as
#2**2=4
# In the last example, we used randomized weights as parameters for the rotation gates, here, we use 3 angles beta0, beta1 and beta2 as parameters
# We get an array of 5 values derived from the 3 angle values which will be used as paramters
def get_angles(x):
    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3] ** 2) / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(np.linalg.norm(x[2:]) / np.linalg.norm(x))

    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])


In [ ]:
# Defining the state preparation, the sequence of CNOT gates and rotation gates, which take the above 5 generated values as paramters
def state_preparation(a):
    qml.RY(a[0], wires=0)

    qml.CNOT(wires=[0, 1])
    qml.RY(a[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[2], wires=1)

    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[4], wires=1)
    qml.PauliX(wires=0)

In [ ]:
# As we can see, we take 4 input values and return 5 values derived from them, these are used as paramters for the rotation gates
x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0], requires_grad=False)
ang = get_angles(x)


@qml.qnode(dev)
def test(angles):
    state_preparation(angles)

    return qml.state()


state = test(ang)

print("x               : ", np.round(x, 6))
print("angles          : ", np.round(ang, 6))
print("amplitude vector: ", np.round(np.real(state), 6))

In [ ]:
# Defining the layer and the cost function 
def layer(layer_weights):
    for wire in range(2):
        qml.Rot(*layer_weights[wire], wires=wire)
    qml.CNOT(wires=[0, 1])


def cost(weights, bias, X, Y):
    predictions = variational_classifier(weights, bias, X.T)
    return square_loss(Y, predictions)


In [ ]:
# Data preprocessing
#Loading the IRIS dataset
# Adding two extra dimensions as padding in order to make the dimensions similar to the state space dimensions of a 2-qubit vector
# After normalizing the vector, we then generate the 5 parameters
data = np.loadtxt(r"C:\Users\dpsso\Desktop\variational_classifier\data\iris_classes1and2_scaled.txt")
X = data[:, 0:2]
print(f"First X sample (original)  : {X[0]}")

# pad the vectors to size 2^2=4 with constant values
padding = np.ones((len(X), 2)) * 0.1
X_pad = np.c_[X, padding]
print(f"First X sample (padded)    : {X_pad[0]}")

# normalize each input
normalization = np.sqrt(np.sum(X_pad**2, -1))
X_norm = (X_pad.T / normalization).T
print(f"First X sample (normalized): {X_norm[0]}")

# the angles for state preparation are the features
features = np.array([get_angles(x) for x in X_norm], requires_grad=False)
print(f"First features sample      : {features[0]}")

Y = data[:, -1]

In [ ]:
# We plot the graphs between different dimensions of the data(total dimensions are 4) to capture relationships between them
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(X[:, 0][Y == 1], X[:, 1][Y == 1], c="b", marker="o", ec="k")
plt.scatter(X[:, 0][Y == -1], X[:, 1][Y == -1], c="r", marker="o", ec="k")
plt.title("Original data")
plt.show()

plt.figure()
dim1 = 0
dim2 = 1
plt.scatter(X_norm[:, dim1][Y == 1], X_norm[:, dim2][Y == 1], c="b", marker="o", ec="k")
plt.scatter(X_norm[:, dim1][Y == -1], X_norm[:, dim2][Y == -1], c="r", marker="o", ec="k")
plt.title(f"Padded and normalised data (dims {dim1} and {dim2})")
plt.show()

plt.figure()
dim1 = 0
dim2 = 3
plt.scatter(features[:, dim1][Y == 1], features[:, dim2][Y == 1], c="b", marker="o", ec="k")
plt.scatter(features[:, dim1][Y == -1], features[:, dim2][Y == -1], c="r", marker="o", ec="k")
plt.title(f"Feature vectors (dims {dim1} and {dim2})")
plt.show()


In [ ]:
# Splitting the dataset into training and testing(75% training and 25% testing)
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
X_train = X[index[:num_train]]
X_val = X[index[num_train:]]


In [ ]:
# Defining the variables-no. of layers and no. of qubits along with weights and biases
num_qubits = 2
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

In [ ]:
# We train our model and minimize the cost
opt = NesterovMomentumOptimizer(0.01)
batch_size = 5

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(60):
    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)

    # Compute predictions on train and validation set
    predictions_train = np.sign(variational_classifier(weights, bias, feats_train.T))
    predictions_val = np.sign(variational_classifier(weights, bias, feats_val.T))

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    if (it + 1) % 2 == 0:
        _cost = cost(weights, bias, features, Y)
        print(
            f"Iter: {it + 1:5d} | Cost: {_cost:0.7f} | "
            f"Acc train: {acc_train:0.7f} | Acc validation: {acc_val:0.7f}"
        )

In [ ]:
# Plotting the data points of -1 class and 1 class and comparing their features according to a graph
# The training and validation sets of data have been outlined to understand their relationships
plt.figure()
cm = plt.cm.RdBu

# make data for decision regions
xx, yy = np.meshgrid(np.linspace(0.0, 1.5, 30), np.linspace(0.0, 1.5, 30))
X_grid = [np.array([x, y]) for x, y in zip(xx.flatten(), yy.flatten())]

# preprocess grid points like data inputs above
padding = 0.1 * np.ones((len(X_grid), 2))
X_grid = np.c_[X_grid, padding]  # pad each input
normalization = np.sqrt(np.sum(X_grid**2, -1))
X_grid = (X_grid.T / normalization).T  # normalize each input
features_grid = np.array([get_angles(x) for x in X_grid])  # angles are new features
predictions_grid = variational_classifier(weights, bias, features_grid.T)
Z = np.reshape(predictions_grid, xx.shape)

# plot decision regions
levels = np.arange(-1, 1.1, 0.1)
cnt = plt.contourf(xx, yy, Z, levels=levels, cmap=cm, alpha=0.8, extend="both")
plt.contour(xx, yy, Z, levels=[0.0], colors=("black",), linestyles=("--",), linewidths=(0.8,))
plt.colorbar(cnt, ticks=[-1, 0, 1])

# plot data
for color, label in zip(["b", "r"], [1, -1]):
    plot_x = X_train[:, 0][Y_train == label]
    plot_y = X_train[:, 1][Y_train == label]
    plt.scatter(plot_x, plot_y, c=color, marker="o", ec="k", label=f"class {label} train")
    plot_x = (X_val[:, 0][Y_val == label],)
    plot_y = (X_val[:, 1][Y_val == label],)
    plt.scatter(plot_x, plot_y, c=color, marker="^", ec="k", label=f"class {label} validation")

plt.legend()
plt.show()

### Summary

We learned to build and train a quantum classifier using a hybrid quantum-classical approach. We began by understanding the theory behind variational quantum circuits and their role in quantum machine learning. We then moved on to implement the classifier with PennyLane and PyTorch, starting with data preparation, followed by constructing a variational circuit, and finally training the model. We also evaluated the classifier's performance on a simple dataset, gaining insights into the practical applications of quantum machine learning.